# TP2 - Benchmark automatique

Dans ce TP nous allons définir une fonction pour mesurer les performances d'un modèle de langage via l'exécution de plusieurs benchmarks. Nous avons vu en cours trois manières de mesurer la performance d'un modèle de langage qu'on peut résumer à:
1. **Évaluation automatique**: via un ensemble de questions dont on connait la réponse
2. **Évaluation humaine**: qualification humaine de la réponse d'un modèle à une question
3. **Évaluation par modèle de langage**: notation ou comparaison de réponse d'un ou plusieurs modèles par un autre modèle

Nous nous intéressons ici au premier point, en particulier avec les benchmarks [GSM8K](https://huggingface.co/datasets/openai/gsm8k) et [HellaSwag](https://huggingface.co/datasets/Rowan/hellaswag).
Dans l'ensemble du notebook nous utiliserons la librairie LangChain.

Il est à garder en tête que ce notebook n'a qu'une portée pédagogique et n'est pas forcément à jour puisque le domaine évolue rapidement, ni que les pratiques sont celles validées par l'industrie.

## Uniformisation des benchmarks

Pour chaque benchmark que l'on considère, nous avons besoin de plusieurs informations :
* **Dataset** : une fonction pour charger les questions du benchmark
* **Référence** : une fonction capable d'identifier la réponse attentue
* **Prompt** : un prompt qui permet de demander correctement au modèle de répondre à la question
* **Chaîne** : une fonction qui renvoie la chaîne de traitement de LangChain
* **Score** : une fonction qui score la performance d'un modèle sur une question

Nous allons commencer par créer une classe qui regroupe ces desiderata :


In [35]:
from langchain_core.runnables import Runnable
from langchain_core.prompts import PromptTemplate


class Benchmark:
    name: str

    def __init__(self, prompt: PromptTemplate):
        self.prompt = prompt

    def load_data(self):
        raise NotImplementedError

    def build_chain(self, model) -> Runnable:
        raise NotImplementedError

    def get_reference(self, sample):
        raise NotImplementedError

    def score(self, prediction, reference):
        raise NotImplementedError

Pour rendre cette classe plus concrète, commençons par travailler avec le benchmark [GSM8K](https://huggingface.co/datasets/openai/gsm8k).

### Benchmark GSM8K

On commence par charger le dataset et observer une question.

**Consigne** : Résoudre la question *à la main* et vérifier votre réponse. On recommande d'explorer plusieurs questions.

In [36]:
import numpy as np; np.random.seed(42)
from datasets import load_dataset

dataset = load_dataset("gsm8k", "main")
dataset = dataset["test"]

print(f"Number of questions: {len(dataset)}")
index = 0
print("Example of question:\n", dataset[index]["question"])
print("And its answer:\n", dataset[index]["answer"])

Number of questions: 1319
Example of question:
 Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
And its answer:
 Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


Après avoir inspecté plusieurs éléments du dataset, on remarque que la réponse finale est placée après la chaîne de caractères "####".

**Consigne**: Construire une fonction `get_reference` qui prend en argument un élément de GMS8K (dictionnaire avec question et réponse) et renvoie la réponse attendue (string). On pourra utiliser la fonction [`search`](https://docs.python.org/3/library/re.html#re.search) de la librairie [`re`](https://docs.python.org/3/library/re.html#).
Puis tester cette fonction sur l'exemple précédent.

In [37]:
from re import search

def get_reference(sample: dict) -> str:
    match = search(r"#### (\d+)", sample["answer"])
    return match.group(1) if match else None

index = 0
reference = get_reference(sample=dataset[index])
print(f"Reference: {reference}")

Reference: 18


Il nous reste maintenant à définir un prompt tel que l'on puisse appeler un modèle et tester notre mécanique.

In [38]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        """You are a careful mathematician. Solve the problem step by step, then display your answer in the end.
        Question: {question}
        Answer:"""
    )
)

En intégrant l'appel à un modèle via Ollama sur notre ordinateur, on peut définir avec LangChain :

In [39]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="gemma3:4b")

chain = (
    {"question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

index = 0

question = dataset[index]["question"]
answer = get_reference(dataset[index])
response = chain.invoke(question)
print(f"Model answer : {response}")
print(f"The answer was : {answer}")


Model answer : Here's how we can solve this problem step by step:

1. **Calculate the total number of eggs laid:** Janet's ducks lay 16 eggs per day.

2. **Calculate the number of eggs eaten:** She eats 3 eggs per day.

3. **Calculate the number of eggs remaining after breakfast:** 16 eggs (laid) - 3 eggs (eaten) = 13 eggs

4. **Calculate the number of eggs used for baking:** She uses 4 eggs for baking.

5. **Calculate the number of eggs remaining after baking:** 13 eggs - 4 eggs (baking) = 9 eggs

6. **Calculate the earnings from selling the remaining eggs:** She sells 9 eggs at $2 per egg.  So she makes 9 * $2 = $18.

**Answer:** $18
The answer was : 18


Il nous faut extraire la dernière valeur numérique pour obtenir automatiquement la réponse du modèle.

**Consigne** : Définir une fonction `score` qui prend en paramètre la réponse du modèle et la réponse attendue puis renvoie si les deux réponses sont identiques (1 / 0). On pourra utiliser la fonction [`findall`](https://docs.python.org/3/library/re.html#re.findall) de la librairie `re`.
Puis l'appliquer sur l'exemple précédent.

In [40]:
from re import findall

def score(prediction, reference):
    if reference is None:
        return 0.0

    numbers = findall(r"\d+", prediction)
    return 1.0 if numbers and numbers[-1] == reference else 0.0

value = score(response, answer)
print(f"The model scored {value}")

The model scored 1.0


Nous avons l'ensemble des éléments nécessaire pour définir la classe `GSM8KBenchmark` depuis la classe `Benchmark` que nous avons défini précédemment.

**Consigne** : Définir cette classe comme sous-classe de `Benchmark`.

In [41]:
class GSM8KBenchmark(Benchmark):
    name = "GSM8K"

    def load_data(self):
        return load_dataset("gsm8k", "main", split="test")

    def build_chain(self, model):
        return (
            {"question": RunnablePassthrough()}
            | self.prompt
            | model
            | StrOutputParser()
        )

    def get_reference(self, sample):
        match = search(r"#### (\d+)", sample["answer"])
        return match.group(1) if match else None

    def score(self, prediction, reference):
        if reference is None:
            return 0.0
        numbers = findall(r"\d+", prediction)
        return 1.0 if numbers and numbers[-1] == reference else 0.0

Il est maintenant temps de définir une fonction qui *fait* le benchmark.

**Consigne** : Définir une fonction `run_benchmark` qui prend en paramètre :
* `model_name` : le nom du modèle Ollama que l'on veut tester
* `benchmark` : la classe benchmark que l'on souhaite tester
* `max_samples` : le nombre maximum de questions que l'on souhaite utiliser

Puisque l'object avec lequel nous travaillons est un dataset HuggingFace, pour sélectionner $n$ lignes, on utilisera 
```python
dataset = dataset.select(range(max_samples))
```
De cette manière on préserve la structure.

In [42]:
from tqdm import tqdm
import numpy as np


def run_benchmark(model_name: str, benchmark: Benchmark, max_samples: int | None = None) -> dict:

    model = OllamaLLM(model=model_name)

    data = benchmark.load_data()
    if max_samples:
        data = data.select(range(max_samples))
    chain = benchmark.build_chain(model)

    scores = []

    for sample in tqdm(data, desc=f"Running {benchmark.name}"):
        prediction = chain.invoke(sample)
        reference = benchmark.get_reference(sample)
        scores.append(benchmark.score(prediction, reference))

    results = {
        "benchmark": benchmark.name,
        "model": model_name,
        "num_samples": len(scores),
        "accuracy": np.mean(scores),
    }
    return results


**Consigne** : Utiliser la fonction `run_benchmark` en définissant un prompt pour GSM8K.

In [43]:
prompt_GMS8K = PromptTemplate(
    input_variables=["question"],
    template=(
        """You are a careful mathematician. Solve the problem step by step, then display your answer in the end.
        Question: {question}
        Answer:"""
        )
    )

benchmark_GSM8K = GSM8KBenchmark(prompt=prompt_GMS8K)
results = run_benchmark(model_name="gemma3:4b", benchmark=benchmark_GSM8K, max_samples=5)
print(results)

Running GSM8K: 100%|██████████| 5/5 [00:50<00:00, 10.18s/it]

{'benchmark': 'GSM8K', 'model': 'gemma3:4b', 'num_samples': 5, 'accuracy': 0.8}


### HellaSwag

Maintenant que nous avons réussi à le faire pour le dataset GMS8K, attaquons-nous à [HellaSwag](https://huggingface.co/datasets/Rowan/hellaswag).

**Consigne** : En suivant la même approche que précédemment, implémenter une sous classe `HellaSwagBenchmark` à partir de la classe `Benchmark`. Puis utiliser la fonction `run_benchmark` pour valider votre travail.

In [44]:
class HellaSwagBenchmark(Benchmark):
    name = "HellaSwag"

    def load_data(self):
        return load_dataset("hellaswag", split="validation")

    def build_chain(self, model):
        return (
            {
                "context": lambda x: x["ctx"],
                "choices": lambda x: "\n".join(
                    f"{index}: {choice}" for index, choice in enumerate(x["endings"])
                ),
            }
            | self.prompt
            | model
            | StrOutputParser()
        )

    def get_reference(self, sample):
        return str(sample["label"])

    def score(self, prediction, reference):
        match = search(r"\d", prediction)
        return 1.0 if match and match.group(0) == reference else 0.0


In [45]:
prompt_HellaSwag = PromptTemplate(
    input_variables=["context", "choices"],
    template=(
        """You will be given a context and then different choices. You need to find the most likely continuation to the context. Answer with the number of the most likely choice only.
        Context: {context}
        Choices: {choices}
        Answer:"""
    )
)

benchmark_HellaSwag = HellaSwagBenchmark(prompt=prompt_HellaSwag)

results = run_benchmark(model_name="gemma3:4b", benchmark=benchmark_HellaSwag, max_samples=5)
print(results)

Running HellaSwag: 100%|██████████| 5/5 [00:02<00:00,  2.08it/s]

{'benchmark': 'HellaSwag', 'model': 'gemma3:4b', 'num_samples': 5, 'accuracy': 1.0}


## Réponses structurées

Sur quelques exemples tout semble fonctionner ! Mais il y a au moins une fragilité dans notre travail : la récupération de la réponse est peu fiable et largement dépendante des prompts.


Par exemple pour GMS8K, on aimerait avoir une réponse sous la forme d'un JSON :
```json
{
  "reasoning": "étapes de raisonnement",
  "final_answer": 18
}
```

De cette manière ce serait particulièrement simple d'extraire la réponse, sans pour autant ne pas avoir de *réflexion* du modèle. En revanche pour HellaSwag, un JSON extrêment simple suffit :
```json
{
    "choice": 2
}
```

Pour forcer le modèle à suivre ces formats, nous allons utiliser l'option [Pydantic](https://docs.langchain.com/oss/python/langchain/structured-output). Elle s'utilise comme suit, pour GSM8K :

In [46]:
from pydantic import BaseModel, Field

class GSM8KOutput(BaseModel):
    reasoning: str = Field(description="Step-by-step reasoning")
    final_answer: float = Field(description="Final numeric answer")


Concernant l'intégration dans le prompt :

In [ ]:
from langchain.output_parsers import PydanticOutputParser

parser_gsm8k = PydanticOutputParser(pydantic_object=GSM8KOutput)

prompt_gsm8k = PromptTemplate(
    input_variables=["question"],
    partial_variables={"format_instructions": parser_gsm8k.get_format_instructions()},
    template=(
        """You are a careful mathematician. Solve the problem step by step.
        Question: {question}
        {format_instructions}"""
    ),
)

print(parser_gsm8k.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"reasoning": {"description": "Step-by-step reasoning", "title": "Reasoning", "type": "string"}, "final_answer": {"description": "Final numeric answer", "title": "Final Answer", "type": "number"}}, "required": ["reasoning", "final_answer"]}
```


**Consigne** : Modifier la classe `Benchmark` et la sous-classe `GMS8KBenchmark` pour intégrer ces évolutions.

In [67]:
from langchain_core.runnables import Runnable
from langchain_core.prompts import PromptTemplate


class Benchmark:
    name: str

    def __init__(self, prompt: PromptTemplate, parser: PydanticOutputParser):
        self.prompt = prompt
        self.parser = parser

    def load_data(self):
        raise NotImplementedError

    def build_chain(self, model) -> Runnable:
        raise NotImplementedError

    def get_reference(self, sample):
        raise NotImplementedError

    def score(self, prediction, reference):
        raise NotImplementedError

In [68]:
class GSM8KBenchmark(Benchmark):
    name = "GSM8K"

    def load_data(self):
        return load_dataset("gsm8k", "main", split="test")

    def build_chain(self, model):
        return (
            {"question": RunnablePassthrough()}
            | self.prompt
            | model
            | self.parser
        )

    def get_reference(self, sample):
        match = search(r"#### (\d+)", sample["answer"])
        return float(match.group(1)) if match else None

    def score(self, prediction: GSM8KOutput, reference: float | None):
        if reference is None:
            return 0.0
        return 1.0 if prediction.final_answer == reference else 0.0

**Consigne** : Utiliser la fonction `run_benchmark` et vérifier que tout fonctionne.

In [69]:
gsm8k = GSM8KBenchmark(
    prompt=prompt_gsm8k,
    parser=parser_gsm8k,
)

run_benchmark("gemma3:4b", gsm8k, max_samples=5)

Running GSM8K: 100%|██████████| 5/5 [01:01<00:00, 12.25s/it]


{'benchmark': 'GSM8K', 'model': 'gemma3:4b', 'num_samples': 5, 'accuracy': 0.8}

**Consigne** : Réaliser la même modification pour HellaSwag, et vérifier que cela fonctionne.

In [63]:
class HellaSwagOutput(BaseModel):
    choice: int = Field(description="Index of the chosen continuation")

class HellaSwagBenchmark(Benchmark):
    name = "HellaSwag"

    def load_data(self):
        return load_dataset("hellaswag", split="validation")

    def build_chain(self, model):
        return (
            {
                "context": lambda x: x["ctx"],
                "choices": lambda x: "\n".join(
                    f"{index}: {choice}" for index, choice in enumerate(x["endings"])
                ),
            }
            | self.prompt
            | model
            | self.parser
        )

    def get_reference(self, sample):
        return str(sample["label"])

    def score(self, prediction: HellaSwagOutput, reference: str) -> float:
        return 1.0 if str(prediction.choice) == reference else 0.0


In [65]:
parser_hellaswag= PydanticOutputParser(pydantic_object=HellaSwagOutput)

prompt_HellaSwag = PromptTemplate(
    input_variables=["context", "choices"],
    partial_variables={"format_instructions": parser_hellaswag.get_format_instructions()},
    template=(
        """You will be given a context and then different choices. You need to find the most likely continuation to the context.
        Context: {context}
        Choices: {choices}
        {format_instructions}"""
    )
)

hella_swag = HellaSwagBenchmark(
    prompt=prompt_HellaSwag,
    parser=parser_hellaswag,
)

run_benchmark("gemma3:4b", hella_swag, max_samples=5)

Running HellaSwag: 100%|██████████| 5/5 [00:15<00:00,  3.12s/it]


{'benchmark': 'HellaSwag',
 'model': 'gemma3:4b',
 'num_samples': 5,
 'accuracy': 1.0}

## Pour aller plus loin

On pourrait implémenter d'autres benchmark, comparer vraiment des modèles entre eux, comparer des prompts entre eux...